In [ ]:
# Notebook from https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7
!pip install -q scipy langchain transformers playwright html2text sentence_transformers faiss-gpu
!pip install -q --upgrade git+https://github.com/huggingface/transformers

In [ ]:
!playwright install > /dev/null
!playwright install-deps > /dev/null

In [ ]:
!pip install -U tokenizers

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig




from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain


#################################################################
# Tokenizer
#################################################################

model_name="mistralai/Mistral-7B-Instruct-v0.1"
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir="."
)
#load_in_4bits=True)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
article_a_indexer="https://www.droits-salaries.com/420531139-gie-auxia-gestion/42053113900079/T07521028203-accord-relatif-au-teletravail-au-sein-du-gie-auxia-gestion-teletravail.shtml"

In [ ]:
import nest_asyncio

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

nest_asyncio.apply()

# Articles to index
articles = [article_a_indexer]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

# Converts HTML to plain text 
html2text = Html2TextTransformer()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(docs[0].page_content, 'html.parser')
new_docs=soup.find(class_="texte_accord").text

In [ ]:
with open("test.txt","w") as file:
    file.write(new_docs)

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./test.txt")
docs=loader.load()

In [ ]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=100)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [ ]:
prompt_template = """

### [ROLE] Role: Vous êtes un expert juridique des accords d'entreprise.


### [INST] Instruction: Votre travail consiste à peupler et mettre à jour une base des données nécessaires pour réaliser des études économétriques poussées sur le télétravail.
        Pour cela, vous devez prendre les textes des accords d'entreprise dans lesquelles figurent les informations à jour que l'on vous fourni, et d'extraire précisément l'information spécifique dont vous avez besoin.
        Vous ne devez extraire que le montant, et uniquement le montant.
        Si vous ne trouver pas la réponse dans le texte vous devez le dire et ne pas chercher à en fournir une autre.
        Avant de répondre, vérifier que la réponse se trouve bien dans le texte indiqué.
        Impératif pour votre raisonnement : Déconnecter les liens entre les périodicités (semaine, mois, trimestre, année)! Faire comme si les semaines, les mois, les trimestres et les années sont des concepts indépendants : dans un mois, il y a donc 0 semaine ; dans un trimestre, 0 mois et 0 semaine; dans une année, 0 trimestre, 0 mois et 0 semaine !
{context}

### QUESTION:
{question} [/INST]
"""

In [ ]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Q_DUREE="S'il est fait mention d'une durée de l'accord, est-ce que l'accord est à durée déterminée ? Oui=1 ou non=0 ou NA? "
Q_REVERS="S'il est fait mention d'une clause et d'un article de réversibilité, est-ce que l'accord a une clause ou un article de réversibilité ? Oui=1 ou non=0 ou NA?"
Q_ADAPT="S'il est fait mention d'une période d'adaptation, est-ce que l'accord comprend une période d'adaptation ? Oui=1 ou non=0 ou NA?"
Q_TTREG="S'il est fait mention d'un télétravail régulier, est-ce que l'accord détaille le télétravail régulier ? Oui=1 ou non=0 ou NA?"
Q_TTOCA="S'il est fait mention d'un télétravail occasionnel, est-ce que l'accord mentionne le télétravail occasionnel ? Oui=1 ou non=0 ou NA?"
Q_TTEXC="S'il est fait mention d'un télétravail exceptionnel, est-ce que l'accord mentionne le télétravail exceptionnel ? Oui=1 ou non=0 ou NA?"
Q_TTSEM="S'il est fait mention d'une quotité par semaine, combien de jours un salarié peut-il télétravailler par semaine ?"
Q_TTMOIS="S'il est fait mention d'une quotité par mois, combien de jours un salarié peut-il télétravailler par mois ?"
Q_TTTRIM="S'il est fait mention d'une quotité par trimestre, combien de jours un salarié peut-il télétravailler par trimestre ?"
Q_TTANNEE="S'il est fait mention d'une quotité par an, combien de jours un salarié peut-il télétravailler par an ?"
Q_EQUIP="S'il est fait mention d'un équipement fourni, est-ce que l'accord mentionne un équipement fourni ?"
Q_COMPJ="S'il est fait mention d'une indemnité journaliere, de combien est l'indemnité forfaitaire par jour de télétravail ?"
Q_COMPM="S'il est fait mention d'une indemnité mensuelle, de combien est l'indemnité mensuelle par mois de télétravail ?"
Q_COMPA="S'il est fait mention d'une indemnité annuelle, de combien est l'indemnité annuelle par année de télétravail ?"
Q_COMPO="S'il est fait mention d'une indemnité autres que journalière, mensuelle ou annuelle, de combien est l'indemnité liée au télétravail ?"
Q_COMPC="If an exceptional allowance is mentioned, what is the maximum exceptional allowance for any remote work for covid?"
Q_COMPE="S'il est fait mention d'une indemnité d'équipement, de combien est l'indemnité d'équipement maximale liée au télétravail ?"

In [ ]:
reponse= rag_chain.invoke(Q_DUREE)
print("Q_DUREE:",reponse["text"])
reponse= rag_chain.invoke(Q_REVERS)
print("Q_REVERS:",reponse["text"])
reponse= rag_chain.invoke(Q_ADAPT)
print("Q_ADAPT:",reponse["text"])
reponse= rag_chain.invoke(Q_TTREG)
print("Q_TTREG:",reponse["text"])
reponse= rag_chain.invoke(Q_TTOCA)
print("Q_TTOCA:",reponse["text"])
reponse= rag_chain.invoke(Q_TTEXC)
print("Q_TTEXC:",reponse["text"])
reponse= rag_chain.invoke(Q_TTSEM)
print("Q_TTSEM:",reponse["text"])
reponse= rag_chain.invoke(Q_TTMOIS)
print("Q_TTMOIS:",reponse["text"])
reponse= rag_chain.invoke(Q_TTTRIM)
print("Q_TTTRIM:",reponse["text"])
reponse= rag_chain.invoke(Q_TTANNEE)
print("Q_TTANNEE:",reponse["text"])
reponse= rag_chain.invoke(Q_EQUIP)
print("Q_EQUIP:",reponse["text"])
reponse= rag_chain.invoke(Q_COMPJ)
print("Q_COMPJ:",reponse["text"])
reponse= rag_chain.invoke(Q_COMPM)
print("Q_COMPM:",reponse["text"])
reponse= rag_chain.invoke(Q_COMPA)
print("Q_COMPA:",reponse["text"])
reponse= rag_chain.invoke(Q_COMPO)
print("Q_COMPO:",reponse["text"])
reponse= rag_chain.invoke(Q_COMPC)
print("Q_COMPC:",reponse["text"])
reponse= rag_chain.invoke(Q_COMPE)
print("Q_COMPE:",reponse["text"])

In [ ]:
Q_TTSEM="Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par semaine ?"
reponse= rag_chain.invoke(Q_TTSEM)
print("Q_TTSEM:",reponse["text"])

In [ ]:
Q_TTSEM_NOMBRE="Sélectionne seulement les paragraphes mentionnant un nombre de jour de télétravail autorisé explicitement exprimé par semaine et s'il existe donne le nombre de jour maximum par semaine sans compter les jours supplémentaires, modulables ou exceptionnels"
reponse= rag_chain.invoke(Q_TTSEM_NOMBRE)
print("Q_TTSEM_NOMBRE:",reponse["text"])

In [ ]:
Q_TTMOIS="Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail autorisé maximum explicitement exprimé par mois ?"
reponse= rag_chain.invoke(Q_TTMOIS)
print("Q_TTMOIS:",reponse["text"])

In [ ]:
Q_TTMOIS_NOMBRE="Sélectionne seulement les paragraphes mentionnant un nombre de jour de télétravail autorisé explicitement exprimé par mois et s'il existe donne le nombre de jour maximum par mois sans compter les jours supplémentaires, modulables ou exceptionnels"
reponse= rag_chain.invoke(Q_TTMOIS_NOMBRE)
print("Q_TTMOIS_NOMBRE:",reponse["text"])

In [ ]:
Q_TTTRIM="Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par trimestre ?"
reponse= rag_chain.invoke(Q_TTTRIM)
print("Q_TTTRIM:",reponse["text"])

In [ ]:
Q_TTTRIM_NOMBRE="Sélectionne seulement les paragraphes mentionnant un nombre de jour de télétravail autorisé explicitement exprimé par trimestre et s'il existe donne le nombre de jour maximum par trimestre sans compter les jours supplémentaires, modulables ou exceptionnels"
reponse= rag_chain.invoke(Q_TTTRIM_NOMBRE)
print("Q_TTTRIM_NOMBRE:",reponse["text"])

In [ ]:
Q_TTANNEE="Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par année civile, noir sur blanc, juridiquement ?"
reponse= rag_chain.invoke(Q_TTANNEE)
print("Q_TTANNEE:",reponse["text"])

In [ ]:
Q_TTANNEE_NOMBRE="Sélectionne seulement les paragraphes mentionnant un nombre de jour de télétravail autorisé explicitement exprimé par année et s'il existe donne le nombre de jour maximum par année sans compter les jours supplémentaires, modulables ou exceptionnels"
reponse= rag_chain.invoke(Q_TTANNEE_NOMBRE)
print("Q_TTANNEE_NOMBRE:",reponse["text"])

In [ ]:
print("Q_TTANNEE_NOMBRE:",reponse["context"])